In [1]:
from surgeon_recording.reader import Reader
from surgeon_recording.functions_metrics import F_metrics

from os.path import join
import csv
import pandas as pd
import numpy as np
from tabulate import tabulate
import scipy.interpolate as inter
from pyquaternion import Quaternion
from numpy import linalg as LA
import plotly.graph_objects as go
import os
import scipy.ndimage
import scipy.signal
import matplotlib.pyplot as plt
import matplotlib
from plotly.subplots import make_subplots
from plotly import subplots
import matplotlib.lines
from scipy import integrate
import math 
from scipy import signal as sp


reader = Reader()
f_metrics = F_metrics()

## READ DATA

In [2]:
path_to_data_folder = '/home/anaelle/git/surgeon_recording/data_paper'                 #lasa computer

subject_list_tot = ['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12']

segment_list = ['500','510','530','540','541']

sensors = ['TPS_calibrated']


data_segments = {}
mean_segments = {}

run_list = {}
headers = {}
file_list = {}


# List of run we take for analysis, for each subject
run_list = {}
for subject in subject_list_tot:
    run_list[subject] = {}

run_list['S1'] = ['R1','R2','R3'] #ONLY FOR TIME not optitrack data
run_list['S2'] = ['R1','R3','R4']
run_list['S3'] = ['R2','R3','R4']
run_list['S4'] = ['R1','R3','R4']
run_list['S5'] = ['R3','R5','R6']
run_list['S6'] = ['R1','R3','R5']
run_list['S7'] = ['R1','R3','R6']
run_list['S8'] = ['R1','R3','R4']
run_list['S9'] = ['R2','R3','R5']
run_list['S10'] = ['R1','R2','R4']
run_list['S11'] = ['R1','R3','R4']
run_list['S12'] = ['R1','R2','R3']


# Create structures
for subject in subject_list_tot:
    data_segments[subject] = {}
    mean_segments[subject]= {}
    file_list[subject] = {}
        
    for segment in segment_list:
        data_segments[subject][segment]= {}
        file_list[subject][segment] = {}
        mean_segments[subject][segment]= {}
                    
        for run in run_list[subject]:
            data_segments[subject][segment][run] = {}
            mean_segments[subject][segment][run]= {}
            file_list[subject][segment][run] = {}

            for sensor in sensors:
                data_segments[subject][segment][run][sensor]= {}
                mean_segments[subject][segment][run][sensor]= []
                file_list[subject][segment][run][sensor] = []


for sensor in sensors:
    headers[sensor] = []
                
for subject in subject_list_tot:   
        for sensor in sensors:
            for segment in segment_list:
                for run in run_list[subject]:
                    data_folder = join('..', 'data_paper','TPS', subject, run )
                    file_list[subject][segment][run][sensor] = join(data_folder, 'segment_nb_' + segment + '_' + sensor +'.csv')

                    file = open(file_list[subject][segment][run][sensor])
                    reader = csv.DictReader(file)
                    headers[sensor] = reader.fieldnames


                    fingers_cal = headers[sensor][15:-1]

                    data = pd.read_csv(file_list[subject][segment][run][sensor])


                    for finger in fingers_cal:                    
                        data_segments[subject][segment][run][sensor][finger]= data[finger]
                    data_segments[subject][segment][run][sensor]['rel_time'] = data[' relative_time']


                  


## interpolation to have 40 Hz

In [49]:

interpolated_data = {}
time_vector_interp = {}

for subject in subject_list_tot:   
    interpolated_data[subject] = {}
    time_vector_interp[subject] = {}

    for segment in segment_list:
        interpolated_data[subject][segment] = {}
        time_vector_interp[subject][segment] = {}
       
        for run in run_list[subject]:
            interpolated_data[subject][segment][run] = {}
            time_vector_interp[subject][segment][run] = {}

            for finger in fingers_cal:
                interpolated_data[subject][segment][run][finger] = {}
                time_vector_interp[subject][segment][run][finger] = {}

for subject in subject_list_tot:   
    for segment in segment_list:
        for run in run_list[subject]:
             for finger in fingers_cal:
                data = list(data_segments[subject][segment][run][sensor][finger])
                time = np.array(data_segments[subject][segment][run][sensor]['rel_time']/1000)
                
                Interpole= inter.interp1d(list(time), list(data),bounds_error=False, fill_value='extrapolate')
                
                time_step = 1/40
                time_vector = np.arange(time.min(),time.max()-time_step,time_step )
                interpolated_data[subject][segment][run][finger] = Interpole(time_vector)
                time_vector_interp[subject][segment][run][finger] = time_vector
                
                

In [48]:
print(len(interpolated_data[subject][segment][run][finger]))
print(len(data))
print(len(time_vector))
print(len(time))
print(time.min())


0
261
0
261
187.073


## Derivation

In [50]:

abs_diff_tps_estimator = {}
for subject in subject_list_tot:   
    abs_diff_tps_estimator[subject] = {}
    for segment in segment_list:
        abs_diff_tps_estimator[subject][segment] = {}
        for run in run_list[subject]:
            abs_diff_tps_estimator[subject][segment][run]= {}
            for finger in fingers_cal:  
                abs_diff_tps_estimator[subject][segment][run][finger] = []
                
                
    

In [51]:
            
for subject in subject_list_tot:
#for subject in ['S4','S5','S6']:

    for segment in segment_list:

        for run in run_list[subject]: 
            
            for finger in fingers_cal:
                
                window_length = 5
                polyorder = 3
                dt = 1/40 
                
                tmp = scipy.signal.savgol_filter(interpolated_data[subject][segment][run][finger], window_length, polyorder, deriv=1, delta=dt, axis=- 1, mode='interp', cval=0.0)
                
                abs_diff_tps_estimator[subject][segment][run][finger]  = abs(tmp)
                
                

In [46]:
print(len(interpolated_data[subject][segment][run][finger]))

0


In [52]:
#act signal
RH_fingers = {
    'S10':[fingers_cal[5],fingers_cal[3] ], #thumb, index
    'S9':[fingers_cal[5],fingers_cal[3] ],
    'S3':[fingers_cal[5]],
    'S4':[fingers_cal[5]],
    'S5':[fingers_cal[5]],
    'S6':[fingers_cal[5],fingers_cal[3]],
    'S7':[fingers_cal[5],fingers_cal[3] ]

    
    }

LH_fingers = {
    'S10':[fingers_cal[2],fingers_cal[0],fingers_cal[1] ], #thumb, index, middle
    'S9':[fingers_cal[2],fingers_cal[0],fingers_cal[1] ],
    'S3':[fingers_cal[2],fingers_cal[0],fingers_cal[1] ],
    'S6':[fingers_cal[2],fingers_cal[0],fingers_cal[1] ],
    'S5':[fingers_cal[2],fingers_cal[1] ],
    'S4':[fingers_cal[2],fingers_cal[0] ],
    'S7':[fingers_cal[2],fingers_cal[0],fingers_cal[1] ] 
    }


subject_list_tps_activity = ['S10','S9','S3','S7','S6','S5','S4']

sum_derivative_RH = {}
sum_derivative_LH = {}

for subject in subject_list_tps_activity:   
    sum_derivative_RH[subject]={}
    sum_derivative_LH[subject]={}
    for segment in segment_list:
        sum_derivative_RH[subject][segment] = {}
        sum_derivative_LH[subject][segment] = {}
        for run in run_list[subject]:
            sum_derivative_RH[subject][segment][run]=[]
            sum_derivative_LH[subject][segment][run]=[]
                


for subject in subject_list_tps_activity:   
    for segment in segment_list:
        for run in run_list[subject]:
            time_current =  np.array(time_vector_interp[subject][segment][run][finger]/1000) #CONVERSION EN SECONDES
            
            sum_derivative_RH[subject][segment][run] = np.zeros(len(time_current))
            sum_derivative_LH[subject][segment][run] = np.zeros(len(time_current))
            for finger in RH_fingers[subject]:
                sum_derivative_RH[subject][segment][run] = sum_derivative_RH[subject][segment][run] + (abs_diff_tps_estimator[subject][segment][run][finger])
                
            for finger in LH_fingers[subject]:
                sum_derivative_LH[subject][segment][run]= sum_derivative_LH[subject][segment][run] + (abs_diff_tps_estimator[subject][segment][run][finger])
                


          

In [ ]:
subject='S5'
print((abs_diff_tps_estimator[subject][segment]))


In [53]:
# metrics for activity measurements
threshold = 0.1


sum_derivative_RH_norm={}
sum_derivative_LH_norm={}
time_derivative_RH={}
time_derivative_LH={}
sum_derivative_RH_pc={}
sum_derivative_LH_pc={}
sum_derivative_LRH_pc= {}
nb_points_derivative_RH={}
nb_points_derivative_LH={}
nb_peaks_derivative_RH={}
nb_peaks_derivative_LH={}

for subject in subject_list_tps_activity:   
    sum_derivative_RH_norm[subject]={}
    sum_derivative_LH_norm[subject]={}
    sum_derivative_RH_pc[subject]={}
    sum_derivative_LH_pc[subject]={}
    sum_derivative_LRH_pc[subject]={}
    time_derivative_RH[subject]={}
    time_derivative_LH[subject]={}
    nb_points_derivative_RH[subject]={}
    nb_points_derivative_LH[subject]={}
    nb_peaks_derivative_RH[subject]={}
    nb_peaks_derivative_LH[subject]={}

    for segment in segment_list:
        sum_derivative_RH_norm[subject][segment] = {}
        sum_derivative_LH_norm[subject][segment] = {}
        sum_derivative_RH_pc[subject][segment] = {}
        sum_derivative_LH_pc[subject][segment] = {}
        sum_derivative_LRH_pc[subject][segment]={}
        time_derivative_RH[subject][segment]={}
        time_derivative_LH[subject][segment]={}
        nb_points_derivative_RH[subject][segment]={}
        nb_points_derivative_LH[subject][segment]={}
        nb_peaks_derivative_RH[subject][segment] = {}
        nb_peaks_derivative_LH[subject][segment] = {}

        for run in run_list[subject]:
            sum_derivative_RH_norm[subject][segment][run]=[]
            sum_derivative_LH_norm[subject][segment][run]=[]
            sum_derivative_RH_pc[subject][segment][run]=[]
            sum_derivative_LH_pc[subject][segment][run]=[]
            sum_derivative_LRH_pc[subject][segment][run] = []
            time_derivative_RH[subject][segment][run]=[]
            time_derivative_LH[subject][segment][run]=[]
            nb_points_derivative_RH[subject][segment][run]=[]
            nb_points_derivative_LH[subject][segment][run]=[]
            nb_peaks_derivative_RH[subject][segment][run]=[]
            nb_peaks_derivative_LH[subject][segment][run]=[]


for subject in subject_list_tps_activity:   
    for segment in segment_list:
        for run in run_list[subject]:
            
            time_current =  np.array(data_segments[subject][segment][run][sensor]['rel_time']/1000) #CONVERSION EN SECONDES
        
            tmp_R = sum_derivative_RH[subject][segment][run]/max(sum_derivative_RH[subject][segment][run]) #normalisation
            tmp_L = sum_derivative_LH[subject][segment][run]/max(sum_derivative_LH[subject][segment][run]) #normalisation
            
            sum_derivative_RH_norm[subject][segment][run] = tmp_R
            sum_derivative_LH_norm[subject][segment][run] = tmp_L
            
            sum_derivative_RH_pc[subject][segment][run] = len(tmp_R[tmp_R > threshold])/len(tmp_R)*100
            sum_derivative_LH_pc[subject][segment][run] = len(tmp_L[tmp_L > threshold])/len(tmp_L)*100
            
            tmp_LR = tmp_R + tmp_L
            sum_derivative_LRH_pc[subject][segment][run] = len(tmp_LR[tmp_LR > threshold])/len(tmp_R)*100
            
            time_derivative_RH[subject][segment][run] = len(tmp_R[tmp_R > threshold])/40
            time_derivative_LH[subject][segment][run] = len(tmp_L[tmp_L > threshold])/40
            
            nb_points_derivative_RH[subject][segment][run] = len(tmp_R[tmp_R > threshold])
            nb_points_derivative_LH[subject][segment][run] = len(tmp_L[tmp_L > threshold])
            
            
            peaks_R = sp.find_peaks(tmp_R, height=0.12, threshold=0.01, distance=15, prominence=None, width=0, wlen=None, rel_height=1, plateau_size=None)
            peaks_L= sp.find_peaks(tmp_L, height=0.15, threshold=0.01, distance=15, prominence=None, width=3, wlen=None, rel_height=3, plateau_size=None)
            
            nb_peaks_derivative_RH[subject][segment][run] = peaks_R
            nb_peaks_derivative_LH[subject][segment][run] = peaks_L
        
            
             
        
             
            


In [11]:
print(nb_peaks_derivative_RH[subject][segment][run] )

86


# METRICS

## A1 Number of fingers movements

In [57]:
A1_R = {}
A1_L = {}
for subject in subject_list_tps_activity:
    A1_R[subject]={}
    A1_L[subject]={}
    for segment in segment_list:     
        A1_R[subject][segment]={}
        A1_L[subject][segment]={}
        for run in run_list[subject]:
            A1_R[subject][segment][run]= len(nb_peaks_derivative_RH[subject][segment][run][1]['peak_heights'])
            A1_L[subject][segment][run]=len(nb_peaks_derivative_LH[subject][segment][run][1]['peak_heights'])


expert_subjects = ['S4','S10','S3']
intermediate_subjects = ['S5','S6']
novice_subjects = ['S7','S9']
category = ['Novice', 'Intermediate', 'Expert']


subject_expertise = {
    'Expert': expert_subjects,
    'Novice': novice_subjects,
    'Intermediate': intermediate_subjects
}


for segment in segment_list: 
    folder = join('..', 'data_paper' )
    rs_csv = join(folder,'ANOVA','A1','A1_data_SEGMENT' + segment +'.csv')
    f = open(rs_csv, 'w')
    writer = csv.writer(f)
    header = ['specialty','val_RH','val_LH']
    writer.writerow(header)
    for cat in category:     
        for subject in subject_expertise[cat]:   
            for run in run_list[subject]:
                row = [cat, A1_R[subject][segment][run], A1_L[subject][segment][run]]
                writer.writerow(row)
               
    f.close() 


## A2 time activity

In [58]:
A2_R = {}
A2_L = {}
for subject in subject_list_tps_activity:
    A2_R[subject]={}
    A2_L[subject]={}
    for segment in segment_list:     
        A2_R[subject][segment]={}
        A2_L[subject][segment]={}
        for run in run_list[subject]:
            A2_R[subject][segment][run]=time_derivative_RH[subject][segment][run]
            A2_L[subject][segment][run]=time_derivative_LH[subject][segment][run]


expert_subjects = ['S4','S10','S3']
intermediate_subjects = ['S5','S6']
novice_subjects = ['S7','S9']
category = ['Novice', 'Intermediate', 'Expert']


subject_expertise = {
    'Expert': expert_subjects,
    'Novice': novice_subjects,
    'Intermediate': intermediate_subjects
}


for segment in segment_list: 
    folder = join('..', 'data_paper' )
    rs_csv = join(folder,'ANOVA','A2','A2_data_SEGMENT' + segment +'.csv')
    f = open(rs_csv, 'w')
    writer = csv.writer(f)
    header = ['specialty','val_RH','val_LH']
    writer.writerow(header)
    for cat in category:     
        for subject in subject_expertise[cat]:   
            for run in run_list[subject]:
                row = [cat, A2_R[subject][segment][run], A2_L[subject][segment][run]]
                writer.writerow(row)
               
    f.close() 


## A3 pc time

In [59]:
A3_R = {}
A3_L = {}
for subject in subject_list_tps_activity:
    A3_R[subject]={}
    A3_L[subject]={}
    for segment in segment_list:     
        A3_R[subject][segment]={}
        A3_L[subject][segment]={}
        for run in run_list[subject]:
            A3_R[subject][segment][run]=sum_derivative_RH_pc[subject][segment][run]
            A3_L[subject][segment][run]=sum_derivative_LH_pc[subject][segment][run]


expert_subjects = ['S4','S10','S3']
intermediate_subjects = ['S5','S6']
novice_subjects = ['S7','S9']
category = ['Novice', 'Intermediate', 'Expert']


subject_expertise = {
    'Expert': expert_subjects,
    'Novice': novice_subjects,
    'Intermediate': intermediate_subjects
}


for segment in segment_list: 
    folder = join('..', 'data_paper' )
    rs_csv = join(folder,'ANOVA','A3','A3_data_SEGMENT' + segment +'.csv')
    f = open(rs_csv, 'w')
    writer = csv.writer(f)
    header = ['specialty','val_RH','val_LH']
    writer.writerow(header)
    for cat in category:     
        for subject in subject_expertise[cat]:   
            for run in run_list[subject]:
                row = [cat, A3_R[subject][segment][run], A3_L[subject][segment][run]]
                writer.writerow(row)
               
    f.close() 




# PLOTS

In [55]:
for subject in subject_list_tot:   
    for segment in segment_list:
        for run in run_list[subject]:
            time_current =  np.array(data_segments[subject][segment][run][sensor]['rel_time']/1000) #CONVERSION EN SECONDES
            
 

            df = pd.DataFrame({
   
                'Derivation, abs value, left middle':abs_diff_tps_estimator[subject][segment][run][fingers_cal[1]],
                'Derivation, abs value, left thumb':abs_diff_tps_estimator[subject][segment][run][fingers_cal[2]],
                'Derivation, abs value, right index':abs_diff_tps_estimator[subject][segment][run][fingers_cal[3]],
                'Derivation, abs value, right middle':abs_diff_tps_estimator[subject][segment][run][fingers_cal[4]],
                'Derivation, abs value, right thumb':abs_diff_tps_estimator[subject][segment][run][fingers_cal[5]]
            })
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=data_segments[subject][segment][run][sensor][fingers_cal[0]], x=time_current-time_current[0],
                    mode='markers', name=fingers_cal[0]))
            fig.add_trace(go.Scatter(y=interpolated_data[subject][segment][run][fingers_cal[0]], x=time_vector_interp[subject][segment][run][fingers_cal[0]] -time_current[0],
                    mode='lines', name=fingers_cal[0] + '_interp'))            
            
            fig.add_trace(go.Scatter(y=data_segments[subject][segment][run][sensor][fingers_cal[1]], x=time_current-time_current[0],
                    mode='markers', name=fingers_cal[1]))
            fig.add_trace(go.Scatter(y=interpolated_data[subject][segment][run][fingers_cal[1]], x=time_vector_interp[subject][segment][run][fingers_cal[1]] -time_current[0],
                    mode='lines', name=fingers_cal[1]+ '_interp'))
            
            fig.add_trace(go.Scatter(y=data_segments[subject][segment][run][sensor][fingers_cal[2]], x=time_current-time_current[0],
                    mode='markers', name=fingers_cal[2]))
            fig.add_trace(go.Scatter(y=interpolated_data[subject][segment][run][fingers_cal[2]], x=time_vector_interp[subject][segment][run][fingers_cal[2]] -time_current[0],
                    mode='lines', name=fingers_cal[2]+ '_interp'))

            fig.add_trace(go.Scatter(y=data_segments[subject][segment][run][sensor][fingers_cal[3]], x=time_current-time_current[0],
                    mode='markers', name=fingers_cal[3]))
            fig.add_trace(go.Scatter(y=interpolated_data[subject][segment][run][fingers_cal[3]], x=time_vector_interp[subject][segment][run][fingers_cal[3]] -time_current[0],
                    mode='lines', name=fingers_cal[3]+ '_interp'))
            
            fig.add_trace(go.Scatter(y=data_segments[subject][segment][run][sensor][fingers_cal[4]], x=time_current-time_current[0],
                    mode='markers', name=fingers_cal[4]))
            fig.add_trace(go.Scatter(y=interpolated_data[subject][segment][run][fingers_cal[4]], x=time_vector_interp[subject][segment][run][fingers_cal[4]] -time_current[0],
                    mode='lines', name=fingers_cal[4]+ '_interp'))
            
            fig.add_trace(go.Scatter(y=data_segments[subject][segment][run][sensor][fingers_cal[5]], x=time_current-time_current[0],
                    mode='markers', name=fingers_cal[5]))
            fig.add_trace(go.Scatter(y=interpolated_data[subject][segment][run][fingers_cal[5]], x=time_vector_interp[subject][segment][run][fingers_cal[5]]-time_current[0],
                    mode='lines', name=fingers_cal[5]+ '_interp'))
            #fig.show()
            
    

            if not os.path.exists(join('..', 'data_paper','PLOTS','TPS', 'RAW' )):
                os.makedirs(join('..', 'data_paper','PLOTS','TPS', 'RAW'))
            fig.write_image(join('..', 'data_paper','PLOTS','TPS', 'RAW', subject +'_' + run + 'segment' + segment  + '_'  + '_raw_cal.png'))
    
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=abs_diff_tps_estimator[subject][segment][run][fingers_cal[0]], x=time_vector_interp[subject][segment][run][fingers_cal[4]] -time_current[0],
                    mode='lines', name='Derivation, abs value, left index'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, left middle'], x=time_vector_interp[subject][segment][run][fingers_cal[4]] -time_current[0],
                    mode='lines', name='Derivation, abs value, left middle'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, left thumb'], x=time_vector_interp[subject][segment][run][fingers_cal[4]] -time_current[0],
                    mode='lines', name='Derivation, abs value, left thumb'))

            fig.add_trace(go.Scatter(y=df['Derivation, abs value, right index'], x=time_vector_interp[subject][segment][run][fingers_cal[4]] -time_current[0],
                    mode='lines', name='Derivation, abs value, right index'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, right middle'], x=time_vector_interp[subject][segment][run][fingers_cal[4]] -time_current[0],
                    mode='lines', name='Derivation, abs value, right middle'))
            fig.add_trace(go.Scatter(y=df['Derivation, abs value, right thumb'], x=time_vector_interp[subject][segment][run][fingers_cal[4]] -time_current[0],
                    mode='lines', name='Derivation, abs value, right thumb'))
            #fig.show()
            
            if not os.path.exists(join('..', 'data_paper','PLOTS','TPS', 'DIFF' )):
                os.makedirs(join('..', 'data_paper','PLOTS','TPS', 'DIFF'))
            fig.write_image(join('..', 'data_paper','PLOTS','TPS', 'DIFF', subject +'_' + run + 'segment' + segment  +'_abs_derivative.png'))
    


In [40]:
print(time_vector_interp[subject][segment][run][fingers_cal[0]])

[253.927 253.952 253.977 ... 286.127 286.152 286.177]


In [56]:
   
for subject in subject_list_tps_activity:   
    for run in run_list[subject]:
        for segment in segment_list:    
            time_current =  np.array(time_vector_interp[subject][segment][run][fingers_cal[0]]/1000) #CONVERSION EN SECONDES


            df = pd.DataFrame({
                    'Activity right hand':sum_derivative_RH[subject][segment][run]/max(sum_derivative_RH[subject][segment][run]),
                    'Activity left hand':sum_derivative_LH[subject][segment][run]/max(sum_derivative_LH[subject][segment][run]),
                    #'peaks RH':nb_peaks_derivative_RH[subject][run][segment][1]['peak_heights'],
                    #'peaks LH':nb_peaks_derivative_LH[subject][run][segment][1]['peak_heights']


            })
            
            fig  = go.Figure()
            fig = make_subplots(rows=2, cols=1) 

            
            fig.add_trace(go.Scatter(y=df['Activity right hand'], x=time_current-time_current[0],
                    mode='lines', name='Right hand'),1,1)
            
            fig.add_trace(go.Scatter(y=nb_peaks_derivative_RH[subject][segment][run][1]['peak_heights'], x=time_current[nb_peaks_derivative_RH[subject][segment][run][0]]-time_current[0],
                     mode = 'markers', name='Right hand peaks, nb: ' + str(len(nb_peaks_derivative_RH[subject][segment][run][1]['peak_heights']))),1,1)


            
            fig.add_trace(go.Scatter(y=df['Activity left hand'], x=time_current-time_current[0],
                    mode='lines', name='Left hand'),2,1)
            fig.add_trace(go.Scatter(y=nb_peaks_derivative_LH[subject][segment][run][1]['peak_heights'], x=time_current[nb_peaks_derivative_LH[subject][segment][run][0]]-time_current[0],
                    mode='markers', name='Left hand peaks, nb: ' + str(len(nb_peaks_derivative_LH[subject][segment][run][1]['peak_heights']))),2,1)

            fig.update_layout(title='Subject: ' + subject + ', Run: ' + str(run) +', Segment:' + str(segment),
                   xaxis_title='Time',
                   yaxis_title='TPS derivative',
                   width=1025,
                   height=500,
                             )

            fig.update_yaxes(range=[0, 1])


            #fig.show()
            



            if not os.path.exists(join('..', 'data_paper','PLOTS','TPS', 'PEAKS' )):
                os.makedirs(join('..', 'data_paper','PLOTS','TPS', 'PEAKS'))
            fig.write_image(join('..', 'data_paper','PLOTS','TPS', 'PEAKS', subject +'_' + run + 'segment' + segment  +'_abs_derivative.png'))
   

In [69]:
for subject in subject_list_tps_activity:   
    for segment in segment_list:
        for run in run_list[subject]:  
            df = pd.DataFrame({
                        'Activity right hand':sum_derivative_RH[subject][segment][run]/max(sum_derivative_RH[subject][segment][run]),
                        'Activity left hand':sum_derivative_LH[subject][segment][run]/max(sum_derivative_LH[subject][segment][run]),
            })

            fig  = go.Figure()
            fig = make_subplots(rows=2, cols=1) 

            fig.add_trace(go.Scatter(y=df['Activity right hand'], x=time_current-time_current[0],
                    mode='lines', name='Right hand'),1,1)


            fig.add_trace(go.Scatter(y=df['Activity left hand'], x=time_current-time_current[0],
                    mode='lines', name='Left hand'),2,1)

            fig.update_layout(title='Subject: ' + subject + ', Run: ' + str(run) +', Segment:' + str(segment),
                   xaxis_title='Time',
                   yaxis_title='TPS derivative',
                   width=1025,
                   height=500,
                             )

            fig.update_yaxes(range=[0, 0.5])


            #fig.show()



            if not os.path.exists(join('..', 'data_paper','PLOTS','TPS', 'Act_signal' )):
                os.makedirs(join('..', 'data_paper','PLOTS','TPS', 'Act_signal' ))
            fig.write_image(join('..', 'data_paper','PLOTS','TPS', 'Act_signal' , 'run'+ str(run) + '_segment' + segment  +'.png'))


